# Introduction to 25 Tips Project

This project explores 25 SQL Tips. Each tip demonstrates SQL capabilities with examples that can be directly applied.

**Legend:**  
🗃️ Classification of the SQL function or feature. 
📝 An explanation for why or when to use the SQL tip. 
⚙️ Example of how it works. 
🖥️ Sample query.

## **📍1. Use ROLLUP to calculate Subtotals and Grand Totals**

🗂️ **`ROLLUP`** is an extension of the **`GROUP BY`** clause.

📝 Allows you to calculate subtotals and a grand total within a single query. Data can be summarized or grouped in levels without needing to write a query for each level of aggregation.

⚙️ If you have an order details table and want to see total sales by CustomerID and ProductID, along with subtotals per customer and a grand total for all sales.

🖥️    
SELECT C


In [2]:
SELECT 

: Msg 102, Level 15, State 1, Line 1
Incorrect syntax near '0xd83d'.

Total execution time: 00:00:00.002

## **📍2. Replace NULL values with COALESCE** 

🗂️ **COALESCE** is a function.

📝 Allows you to substitute `NULL` values with a value that reads better or understood better by the end user. This is helpful in reports or result sets where `NULL` values might otherwise cause confusion.

⚙️ Suppose you want to list customer names along with their region, but for some customers, the region field is empty (NULL). You can replace these NULL values with "No Region" to make the data moe readable.

🖥️

SELECT 

In [3]:
SELECT


: Msg 102, Level 15, State 1, Line 1
Incorrect syntax near 'SELECT'.

Total execution time: 00:00:00

_All queries use the Northwind\_2023 database._

_All emojis in this project are sourced from [emojicopy.com](https:\emojicopy.com\)_